### MOST UPDATED - 09.09.2022

In [1]:
from glob import glob
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import xgcm
from xorca.lib import load_xorca_dataset
import pickle
import eddytools as et
from cmocean import cm
from dask.diagnostics import ProgressBar
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import timedelta
import matplotlib.colors as colors
import seawater as sw
import math
import scipy.stats as stats
from matplotlib import axes


import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

### path to data

In [ ]:
datapath = '/scratch/usr/shkifmmp/manusc/data/'

In [3]:
def create_stat(model, year_start, year_finished = 2018): # UPDATED 09.09.2022 - > ONLY TYPED0
    
    """ 
        model == 0 > EXP05 elif == 1 EXP06
        typed == 0 > ACC / elif == 1 WHOLE REGION
    """
    
    folder = 'EXP05' if model == 0 else 'EXP06'
    
#     if (typed == 0) & (model == 0):
#         with open(path_EXP05_old + 'track_dict_19580101_' + str(year_finished) + '1231_tracks_typed' + str(typed) 
#                   + '.pickle', 'rb') as f:
#             tracks = pickle.load(f)
#         f.close()
#     else:
    with open(datapath + folder + '/detected/track_dict_' + str(year_start) + '0101_' + str(year_finished) + '1231_tracks_typed0.pickle', 'rb') as f:
        tracks = pickle.load(f)
    f.close()     
    
    eddy_stat = pd.DataFrame.from_dict(tracks, orient = 'index').drop(['eddy_j', 'eddy_i'], axis=1)
    
    for i in eddy_stat.index:
        eddy_stat.loc[i, 'life_time'] = pd.to_datetime(eddy_stat.time.loc[i][-1]) - pd.to_datetime(eddy_stat.time.loc[i][0])
    
    eddy_stat = eddy_stat.loc[(eddy_stat['exist_at_start'] ==  False) & (eddy_stat.terminated == True)]
    
    for i in eddy_stat.index:
        lat = [eddy_stat.loc[i, 'lat'][0], eddy_stat.loc[i, 'lat'][-1]]
        lon = [eddy_stat.loc[i, 'lon'][0], eddy_stat.loc[i, 'lon'][-1]]
        eddy_stat.loc[i, 'distance'] = sw.extras.dist(lat,lon)[0]
        #eddy_stat.loc[i, 'speed'] = eddy_stat.loc[i, 'distance'] / eddy_stat.loc[i, 'life_time']
        eddy_stat.loc[i, 'speed'] = (eddy_stat.loc[i, 'distance'] *1e5)/ (eddy_stat.loc[i, 'life_time'].days * 24 * 3600) # cm/s
#         eddy_stat.loc[i, 'angle'] = (sw.extras.dist(eddy_stat.loc[i, 'lat'], eddy_stat.loc[i, 'lon'])[1]).mean()
        eddy_stat.loc[i, 'area_mean'] = eddy_stat.loc[i, 'area'].mean()
        eddy_stat.loc[i, 'area_max'] = eddy_stat.loc[i, 'area'].max()
        eddy_stat.loc[i, 'amp_mean'] = eddy_stat.loc[i, 'amp'].mean()
        eddy_stat.loc[i, 'amp_max'] = eddy_stat.loc[i, 'amp'].max()
        eddy_stat.loc[i, 'scale_mean'] = eddy_stat.loc[i, 'scale'].mean()
        eddy_stat.loc[i, 'scale_min'] = eddy_stat.loc[i, 'scale'].min()
        eddy_stat.loc[i, 'scale_max'] = eddy_stat.loc[i, 'scale'].max()
        eddy_stat.loc[i, 'year_i'] = pd.to_datetime(eddy_stat.time.loc[i][0]).year
        eddy_stat.loc[i, 'year_f'] = pd.to_datetime(eddy_stat.time.loc[i][-1]).year
    
    eddy_stat = eddy_stat.drop(labels=['lon','lat','amp','area','scale','exist_at_start','terminated'],axis=1)

    eddy_stat = eddy_stat.loc[(eddy_stat['life_time'].astype('timedelta64[D]') >= 30)]
    eddy_stat.loc[:, 'life_time'] = eddy_stat.loc[:, 'life_time'].astype('timedelta64[D]')
        
    eddy_stat.loc[:,'scale_mean'] *= 2
    eddy_stat.loc[:,'scale_max'] *= 2

    eddy_stat = eddy_stat.drop('time', axis=1)

    
    eddy_stat.to_pickle(path= datapath + folder + '/detected/stat_' + folder + '_acc_'+ str(year_finished) + '_' + str(year_finished) +'.pkl')

    return eddy_stat

In [ ]:
create_stat(model=0, year_start=1993, year_finished=2018)